# 1. Imports

In [1]:
import pandas as pd
import spotipy
import warnings
from spotipy.oauth2 import SpotifyOAuth
from config import (
    SPOTIPY_CLIENT_ID,
    SPOTIPY_CLIENT_SECRET,
    SPOTIPY_REDIRECT_URI,
    PLAYLIST_ID
)

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# 2. Consuming _spotipy_ Data

In [2]:
# Creating authentication object
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET,
        redirect_uri=SPOTIPY_REDIRECT_URI,
        scope="playlist-read-private",
    )
)

# Obtaining platlist information
playlist = sp.playlist(PLAYLIST_ID)

print(f'Playlist name: {playlist["name"]}')
music_data = []
for idx, track in enumerate(playlist["tracks"]["items"], start=1):
    track_id = track["track"]["id"]

    # Audio attributes
    track_info = sp.audio_features([track_id])

    # Getting music attributes
    track_details = sp.track(track_id)
    artist = track_details["artists"][0]["name"]
    genre = track_details["genres"] if "genres" in track_details else None
    popularity = track_details["popularity"]

    track_features = track_info[0] if track_info else None

    # Concatenating music infomation
    music_data.append(
        {
            **{
                "track_name": track["track"]["name"],
                "artist": artist,
                "genre": genre,
                "release_date": track_details['album']['release_date'],
                "popularity": popularity,
            },
            **track_features,
        }
    )
data = pd.DataFrame(music_data)

Playlist name: Playlist - Trap


# 3. Data Analysis

In [3]:
data.tail()

,track_name,artist,genre,release_date,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
95,BEM MELHOR,MC Cabelinho,None,2023-05-04,79,0.503,0.587,10,-5.429,0,0.381,0.6130,0.0,0.118,0.566,70.944,audio_features,7aOBf2xYbux3bo1fwISo6b,spotify:track:7aOBf2xYbux3bo1fwISo6b,https://api.spotify.com/v1/tracks/7aOBf2xYbux3...,https://api.spotify.com/v1/audio-analysis/7aOB...,317340,4
96,FOGO E GASOLINA,MC Cabelinho,None,2022-12-20,80,0.531,0.629,10,-6.378,0,0.108,0.1840,0.0,0.134,0.508,141.562,audio_features,52EYWdjiulHaehpexuowyh,spotify:track:52EYWdjiulHaehpexuowyh,https://api.spotify.com/v1/tracks/52EYWdjiulHa...,https://api.spotify.com/v1/audio-analysis/52EY...,181768,4
97,Tropa do Sábio,Mc Poze do Rodo,None,2022-11-10,55,0.643,0.592,1,-6.061,0,0.287,0.3760,0.0,0.106,0.430,146.471,audio_features,75JStwKgd9AMAy4q1SEZjq,spotify:track:75JStwKgd9AMAy4q1SEZjq,https://api.spotify.com/v1/tracks/75JStwKgd9AM...,https://api.spotify.com/v1/audio-analysis/75JS...,146939,4
98,Segredo,KayBlack,None,2023-03-31,83,0.799,0.490,7,-6.788,0,0.178,0.3320,0.0,0.106,0.662,86.272,audio_features,6hu4HvpNzrW2RIEEAZC3vD,spotify:track:6hu4HvpNzrW2RIEEAZC3vD,https://api.spotify.com/v1/tracks/6hu4HvpNzrW2...,https://api.spotify.com/v1/audio-analysis/6hu4...,104145,3
99,Sal e Pimenta,KayBlack,None,2023-03-30,75,0.609,0.530,9,-6.861,0,0.411,0.0434,0.0,0.101,0.278,118.152,audio_features,6BsKZXjKIrGWMK1uLWfJYN,spotify:track:6BsKZXjKIrGWMK1uLWfJYN,https://api.spotify.com/v1/tracks/6BsKZXjKIrGW...,https://api.spotify.com/v1/audio-analysis/6BsK...,206452,5


In this dataframe, we have some columns that we can remove. For example: `uri`, `track_href`, so on...

In [4]:
selected_cols = [
    'id',
    'track_name',
    'artist',
    'genre',
    'release_date',
    'popularity',
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'duration_ms',
    'time_signature'
]
data = data.filter(selected_cols)

In [5]:
data['genre'].unique()

array([None], dtype=object)

For my playlist, the column `genre` is completly empty.